## Training a Custom MobileNet Model

In [1]:
import os

import numpy as np

import keras
from keras.applications import mobilenet_v2, imagenet_utils
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator

import tensorflow

Using TensorFlow backend.


In [2]:
np.random.seed(4096)
tensorflow.set_random_seed(4096)

In [3]:
IMAGE_W = 224
IMAGE_H = 224

INITIAL_LR = 0.0001
EPOCHS = 5
BATCH_SIZE = 32

In [ ]:
image_generator = ImageDataGenerator(
    validation_split=0.2,
    preprocessing_function=mobilenet_v2.preprocess_input,
)

training_generator = image_generator.flow_from_directory(
    'train_images',
    target_size=(IMAGE_W, IMAGE_H),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset='training'
)

validation_generator = image_generator.flow_from_directory(
    'train_images',
    target_size=(IMAGE_W, IMAGE_H),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset='validation'
)

In [ ]:
mobilenet_model = mobilenet_v2.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(IMAGE_W,IMAGE_H,3)
)

mobilenet_model.trainable = False

model = Sequential([
    mobilenet_model,
    GlobalAveragePooling2D(),
#    Dense(1024, activation="relu"),
#    Dense(1024, activation="relu"),
    Dense(512, activation="relu"),
    Dense(training_generator.num_classes, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer = RMSprop(lr=INITIAL_LR, decay=INITIAL_LR/EPOCHS),
    metrics=["accuracy"]
)

H = model.fit_generator(
    training_generator,
    validation_data=validation_generator,
    steps_per_epoch=training_generator.samples / training_generator.batch_size,
    validation_steps=validation_generator.samples / validation_generator.batch_size,
    epochs=EPOCHS
)

model.save('classifier.h5')

## Evaluate Performance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure(figsize=(16,10))
plt.plot(N, H.history["loss"], label="Training Loss")
plt.plot(N, H.history["val_loss"], label="Validation Loss")
plt.plot(N, H.history["acc"], label="Training Accuracy")
plt.plot(N, H.history["val_acc"], label="Validation Accuracy")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Loss / Accuracy")
plt.legend()
plt.show()

In [ ]:
loss, accuracy = model.evaluate_generator(
    validation_generator,
    steps=validation_generator.samples / validation_generator.batch_size
)

print(f"Loss = {loss:0.5f}, Accuracy = {accuracy*100:0.2f}%")

## Notes

### Possible Improvements

- Try without augmentation, or with less
- Try with fewer layers
- Compensate for differing class sizes

### Results

| Accuracy | Loss | Epochs | Optimiser | Image Dimensions | LR | Layers | Other |
|----------|------|:------:|:---------:|:----------------:|----|:-------|:------|
| 71.63%   | 3.1745 | 5    | RMSprop   | 224x224          | 0.0001 | 1024=>1024=>512 | Training accuracy climbed steadily to 98.19%, but validation accuracy decreased from 79% to 71% 
